In [ ]:
"""
SIMULATION OVERVIEW

iRobot is a company (started by MIT alumni and faculty) that sells the Roomba vacuuming robot (watch one of the product videos 
to see these robots in action). Roomba robots move around the floor, cleaning the area they pass over.
In this problem set, you will code a simulation to compare how much time a group of Roomba-like robots will take to clean the 
floor of a room using two different strategies. The following simplified model of a single robot moving in a square 5x5 room 
should give you some intuition about the system we are simulating. The robot starts out at some random position in the room, 
and with a random direction of motion. The illustrations below show the robot's position (indicated by a black dot) as well as 
its direction (indicated by the direction of the red arrowhead).

Time t = 0: The robot starts at the position (2.1, 2.2) with an angle of 205 degrees (measured clockwise from "north"). The tile 
that it is on is now clean.
     t = 1: The robot has moved 1 unit in the direction it was facing, to the position (1.7, 1.3), cleaning another tile.
     t = 2: The robot has moved 1 unit in the same direction (205 degrees from north), to the position (1.2, 0.4), cleaning 
     another tile.
     t = 3: The robot could not have moved another unit in the same direction without hitting the wall, so instead it turns to 
     face in a new, random direction, 287 degrees.
     t = 4: The robot moves along its new direction to the position (0.3, 0.7), cleaning another tile.

Simulation Details

Here are additional details about the simulation model. Read these carefully.

- Multiple robots: In general, there are N > 0 robots in the room, where N is given. For simplicity, assume that robots are 
  points and can pass through each other or occupy the same point without interfering.
- The room: The room is rectangular with some integer width w and height h, which are given. Initially the entire floor is 
  dirty. 
  A robot cannot pass through the walls of the room. A robot may not move to a point outside the room.
- Tiles: You will need to keep track of which parts of the floor have been cleaned by the robot(s). We will divide the area of 
  the   room into 1x1 tiles (there will be w * h such tiles). When a robot's location is anywhere in a tile, we will consider 
  the entire tile to be cleaned (as in the pictures above). By convention, we will refer to the tiles using ordered pairs of 
  integers: (0, 0), (0, 1), ..., (0, h-1), (1, 0), (1, 1), ..., (w-1, h-1).
- Robot motion rules: Each robot has a position inside the room. We'll represent the position using coordinates (x, y) which are 
  floats satisfying 0 ≤ x < w and 0 ≤ y < h. In our program we'll use instances of the Position class to store these coordinates.
  A robot has a direction of motion. We'll represent the direction using an integer d satisfying 0 ≤ d < 360, which gives an 
  angle in degrees. All robots move at the same speed s, a float, which is given and is constant throughout the simulation. 
  Every time-step, a robot moves in its direction of motion by s units. If a robot detects that it will hit the wall within the 
  time-step,that time step is instead spent picking a new direction at random. The robot will attempt to move in that direction 
  on the next time step, until it reaches another wall.
- Termination: The simulation ends when a specified fraction of the tiles in the room have been cleaned.


INTRODUCTION

In this problem set you will practice designing a simulation and implementing a program that uses classes.

As with previous problem sets, please don't be discouraged by the apparent length of this assignment. There is quite a bit to 
read and understand, but most of the problems do not involve writing much code.

ProblemSet2.zip: A zip file of all the files you need, including:

ps2.py, a skeleton of the solution.
ps2_visualize.py, code to help you visualize the robot's movement (an optional - but cool! - part of this problem set).
ps2_verify_movement27.pyc, precompiled module for Python 2.7 that assist with the visualization code.
"""

In [2]:
"""
Problem 1: RectangularRoom Class

You will need to design two classes to keep track of which parts of the room have been cleaned as well as the position and 
direction of each robot. In ps2.py, we've provided skeletons for the following two classes, which you will fill in in Problem 1:

- RectangularRoom: Represents the space to be cleaned and keeps track of which tiles have been cleaned.
- Robot: Stores the position and direction of a robot.
- Position: Stores the x- and y-coordinates of a robot in a room.

Read ps2.py carefully before starting, so that you understand the provided code and its capabilities.

PROBLEM 1

In this problem you will implement two classes, RectangularRoom on this page and Robot on the next. For the RectangularRoom 
class, decide what fields you will use and decide how the following operations are to be performed:

- Initializing the object
- Marking an appropriate tile as cleaned when a robot moves to a given position (casting floats to ints - and/or the function 
  math.floor - may be useful to you here)
- Determining if a given tile has been cleaned
- Determining how many tiles there are in the room
- Determining how many cleaned tiles there are in the room
- Getting a random position in the room
- Determining if a given position is in the room

Complete the RectangularRoom class by implementing its methods in ps2.py. Although this problem has many parts, it should not 
take long once you have chosen how you wish to represent your data. For reasonable representations, a majority of the methods 
will require only a couple of lines of code.)

Hint:
During debugging, you might want to use random.seed(0) so that your results are reproducible.

Enter your code for RectangularRoom below.
"""
# ps2.py

import math
import random

import ps2_visualize
import pylab

# For Python 2.7:
from ps2_verify_movement27 import testRobotMovement

# If you get a "Bad magic number" ImportError, you are not using 
# Python 2.7 and using most likely Python 2.6:

# === Provided class Position
class Position(object):
    """
    A Position represents a location in a two-dimensional room.
    """
    def __init__(self, x, y):
        """
        Initializes a position with coordinates (x, y).
        """
        self.x = x
        self.y = y
        
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def getNewPosition(self, angle, speed):
        """
        Computes and returns the new Position after a single clock-tick has
        passed, with this object as the current position, and with the
        specified angle and speed.

        Does NOT test whether the returned position fits inside the room.

        angle: number representing angle in degrees, 0 <= angle < 360
        speed: positive float representing speed

        Returns: a Position object representing the new position.
        """
        old_x, old_y = self.getX(), self.getY()
        angle = float(angle)
        # Compute the change in position
        delta_y = speed * math.cos(math.radians(angle))
        delta_x = speed * math.sin(math.radians(angle))
        # Add that to the existing position
        new_x = old_x + delta_x
        new_y = old_y + delta_y
        return Position(new_x, new_y)

    def __str__(self):  
        return "(%0.2f, %0.2f)" % (self.x, self.y)


# === Problem 1
class RectangularRoom(object):
    """
    A RectangularRoom represents a rectangular region containing clean or dirty
    tiles.

    A room has a width and a height and contains (width * height) tiles. At any
    particular time, each of these tiles is either clean or dirty.
    """
    def __init__(self, width, height):
        """
        Initializes a rectangular room with the specified width and height.

        Initially, no tiles in the room have been cleaned.

        width: an integer > 0
        height: an integer > 0
        """
        self.width = width
        self.height = height
        self.tiles = {}
         
    def cleanTileAtPosition(self, pos):
        """
        Mark the tile under the position POS as cleaned.

        Assumes that POS represents a valid position inside this room.

        pos: a Position
        """
        x = int(pos.getX())
        y = int(pos.getY())
        if not self.tiles.has_key((x, y)):
            self.tiles[(x,y)] = ''

    def isTileCleaned(self, m, n):
        """
        Return True if the tile (m, n) has been cleaned.

        Assumes that (m, n) represents a valid tile inside the room.

        m: an integer
        n: an integer
        returns: True if (m, n) is cleaned, False otherwise
        """
        if self.tiles.has_key((m, n)):
            return True
        return False
    
    def getNumTiles(self):
        """
        Return the total number of tiles in the room.

        returns: an integer
        """
        return self.width*self.height

    def getNumCleanedTiles(self):
        """
        Return the total number of clean tiles in the room.

        returns: an integer
        """
        if self.tiles:
            return len(self.tiles)
        return 0

    def getRandomPosition(self):
        """
        Return a random position inside the room.

        returns: a Position object.
        """
        x = random.randint(0, self.width-1)
        y = random.randint(0, self.height-1)
        return Position(x, y)

    def isPositionInRoom(self, pos):
        """
        Return True if pos is inside the room.

        pos: a Position object.
        returns: True if pos is in the room, False otherwise.
        """
        import math
        x = int(math.floor(pos.getX()))
        y = int(math.floor(pos.getY()))
        if 0 <= x < self.width and 0 <= y < self.height:
            return True
        return False
    
#room = RectangularRoom(5,4)
#assert not room.isPositionInRoom(Position(-0.10, 0.00)), 'ERROR: isPositionInRoom()'
#room.cleanTileAtPosition(Position(1.23, 0.00))
#assert room.getNumCleanedTiles() == 1, 'ERROR: cleanTileAtPosition() or getNumCleanedTiles()'
#assert room.isTileCleaned(1, 0), 'ERROR: cleanTileAtPosition() or isTileCleaned()'
#random.seed(0)
#pos = room.getRandomPosition()
#assert isinstance(pos, Position), 'ERROR: getRandomPosition()'
#x = pos.getX()
#y = pos.getY()
#assert x == 4.00 and y == 3.00, 'ERROR: getRandomPosition()'

In [3]:
"""
Problem 1: Robot Class

For the Robot class, decide what fields you will use and decide how the following operations are to be performed:

- Initializing the object
- Accessing the robot's position
- Accessing the robot's direction
- Setting the robot's position
- Setting the robot's direction

Complete the Robot class by implementing its methods in ps2.py. When a Robot is initialized, it should clean the first tile it 
is initialized on. Generally the model these Robots will follow is that after a robot lands on a given tile, we will mark the 
entire tile as clean. This might not make sense if you're thinking about really large tiles, but as we make the size of the 
tiles smaller and smaller, this does actually become a pretty good approximation.

Although this problem has many parts, it should not take long once you have chosen how you wish to represent your data. For 
reasonable representations, a majority of the methods will require only a couple of lines of code.)

Note:
The Robot class is an abstract class, which means that we will never make an instance of it. Read up on the Python docs on 
abstract classes at this link and if you want more examples on abstract classes, follow this link. If you took edX course 
6.00.1x already, you've seen an abstract class - the Trigger class from the final problem set!

In the final implementation of Robot, not all methods will be implemented. Not to worry -- its subclass(es) will implement 
the method updatePositionAndClean() (this is similar to the evaluate method of the Trigger class from 6.00.1x ).
"""
class Robot(object):
    """
    Represents a robot cleaning a particular room.

    At all times the robot has a particular position and direction in the room.
    The robot also has a fixed speed.

    Subclasses of Robot should provide movement strategies by implementing
    updatePositionAndClean(), which simulates a single time-step.
    """
    def __init__(self, room, speed):
        """
        Initializes a Robot with the given speed in the specified room. The
        robot initially has a random direction and a random position in the
        room. The robot cleans the tile it is on.

        room:  a RectangularRoom object.
        speed: a float (speed > 0)
        """
        self.room = room
        self.speed = speed
        self.direction = random.randint(0,360)
        self.position = room.getRandomPosition()
        self.room.cleanTileAtPosition(self.position)

    def getRobotPosition(self):
        """
        Return the position of the robot.

        returns: a Position object giving the robot's position.
        """
        return self.position
    
    def getRobotDirection(self):
        """
        Return the direction of the robot.

        returns: an integer d giving the direction of the robot as an angle in
        degrees, 0 <= d < 360.
        """
        return self.direction

    def setRobotPosition(self, position):
        """
        Set the position of the robot to POSITION.

        position: a Position object.
        """
        if self.room.isPositionInRoom(position):
            self.position = position

    def setRobotDirection(self, direction):
        """
        Set the direction of the robot to DIRECTION.

        direction: integer representing an angle in degrees
        """
        self.direction = direction

    def updatePositionAndClean(self):
        """
        Simulate the raise passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        """
        raise NotImplementedError # don't change this!
        
random.seed(0)
room = RectangularRoom(5,5)
eve = Robot(room, 1)
assert isinstance(eve.getRobotPosition(), Position), 'ERROR: getRobotPosition()'
assert eve.getRobotDirection() == 304, 'ERROR: getRobotPosition()'
pos = Position(2,2)
eve.setRobotPosition(pos)
newpos = eve.getRobotPosition()
x = newpos.getX()
y = newpos.getY()
assert x == 2.00 and y == 2.00, 'ERROR: setRobotPosition()'
eve.setRobotPosition(Position(2,5))
newpos = eve.getRobotPosition()
x = newpos.getX()
y = newpos.getY()
assert x == 2.00 and y == 2.00, 'ERROR: setRobotPosition()'
eve.setRobotPosition(Position(5,2))
newpos = eve.getRobotPosition()
x = newpos.getX()
y = newpos.getY()
assert x == 2.00 and y == 2.00, 'ERROR: setRobotPosition()'
eve.setRobotDirection(200)
assert eve.getRobotDirection() == 200, 'ERROR: setRobotDirection()'

In [4]:
"""
Problem 2: StandardRobot Class

Each robot must also have some code that tells it how to move about a room, which will go in a method called 
updatePositionAndClean. Ordinarily we would consider putting all the robot's methods in a single class. However, later in this 
problem set we'll consider robots with alternate movement strategies, to be implemented as different classes with the same 
interface. These classes will have a different implementation of updatePositionAndClean but are for the most part the same as 
the original robots. Therefore, we'd like to use inheritance to reduce the amount of duplicated code.

We have already refactored the robot code for you into two classes: the Robot class you completed in Problem 1 (which contains
general robot code), and a StandardRobot class that inherits from it (which contains its own movement strategy).

Complete the updatePositionAndClean method of StandardRobot to simulate the motion of the robot after a single time-step (as 
described on the Simulation Overview page).


class StandardRobot(Robot):
    ""
    A StandardRobot is a Robot with the standard movement strategy.

    At each time-step, a StandardRobot attempts to move in its current direction; when
    it hits a wall, it chooses a new direction randomly.
    ""
    def updatePositionAndClean(self):
        ""
        Simulate the passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        ""
        
        
We have provided the getNewPosition method of Position, which you may find helpful:


class Position(object):
    def getNewPosition(self, angle, speed):
        ""
        Computes and returns the new Position after a single clock-tick has
        passed, with this object as the current position, and with the
        specified angle and speed.

        Does NOT test whether the returned position fits inside the room.

        angle: number representing angle in degrees, 0 <= angle < 360
        speed: positive float representing speed

        Returns: a Position object representing the new position.
        ""
        
        
Note: You can pass in an integer or a float for the angle parameter. Before moving on to Problem 3, check that your 
implementation of Standard Robot works by uncommenting the following line under your implementation of StandardRobot. Make 
sure that as your robot moves around the room, the tiles it traverses switch colors from gray to white. It should take about 
a minute for it to clean all the tiles.

testRobotMovement(StandardRobot, RectangularRoom)
"""
# === Problem 2
class StandardRobot(Robot):
    """
    A StandardRobot is a Robot with the standard movement strategy.

    At each time-step, a StandardRobot attempts to move in its current
    direction; when it would hit a wall, it *instead* chooses a new direction
    randomly.
    """
    def updatePositionAndClean(self):
        """
        Simulate the raise passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        """
        position = self.getRobotPosition()
        direction = self.getRobotDirection()
        newPosition = position.getNewPosition(direction, self.speed)
        if self.room.isPositionInRoom(newPosition):
            self.setRobotPosition(newPosition)
            self.room.cleanTileAtPosition(newPosition)
        else:
            newDirection = random.randint(0,360)
            self.setRobotDirection(newDirection)
            
            
testRobotMovement(StandardRobot, RectangularRoom)

In [5]:
"""
Problem 3: Running the Simulation

In this problem you will write code that runs a complete robot simulation. Recall that in each trial, the objective is to 
determine how many time-steps are on average needed before a specified fraction of the room has been cleaned. Implement the 
following function:


def runSimulation(num_robots, speed, width, height, min_coverage, num_trials, robot_type):
    ""
    Runs NUM_TRIALS trials of the simulation and returns the mean number of
    time-steps needed to clean the fraction MIN_COVERAGE of the room.

    The simulation is run with NUM_ROBOTS robots of type ROBOT_TYPE, each with
    speed SPEED, in a room of dimensions WIDTH x HEIGHT.
    ""
    

The first six parameters should be self-explanatory. For the time being, you should pass in StandardRobot for the robot_type 
parameter, like so:

avg = runSimulation(10, 1.0, 15, 20, 0.8, 30, StandardRobot)

Then, in runSimulation you should use robot_type(...) instead of StandardRobot(...) whenever you wish to instantiate a robot. 
(This will allow us to easily adapt the simulation to run with different robot implementations, which you'll encounter in 
Problem 5.). Feel free to write whatever helper functions you wish.

We have provided the getNewPosition method of Position, which you may find helpful:


class Position(object):
    def getNewPosition(self, angle, speed):
        ""
        Computes and returns the new Position after a single clock-tick has
        passed, with this object as the current position, and with the
        specified angle and speed.

        Does NOT test whether the returned position fits inside the room.

        angle: integer representing angle in degrees, 0 <= angle < 360
        speed: positive float representing speed

        Returns: a Position object representing the new position.
        ""
        

For your reference, here are some approximate room cleaning times. These times are with a robot speed of 1.0.

- One robot takes around 150 clock ticks to completely clean a 5x5 room.
- One robot takes around 190 clock ticks to clean 75% of a 10x10 room.
- One robot takes around 310 clock ticks to clean 90% of a 10x10 room.
- One robot takes around 3322 clock ticks to completely clean a 20x20 room.
- Three robots take around 1105 clock ticks to completely clean a 20x20 room.

(These are only intended as guidelines. Depending on the exact details of your implementation, you may get times slightly 
different from ours.) You should also check your simulation's output for speeds other than 1.0. One way to do this is to take
the above test cases, change the speeds, and make sure the results are sensible.

For further testing, see the next page in this problem set about the optional way to use visualization methods. Visualization 
will help you see what's going on in the simulation and may assist you in debugging your code.
"""
# === Problem 3
def runSimulation(num_robots, speed, width, height, min_coverage, num_trials, robot_type):
    """
    Runs NUM_TRIALS trials of the simulation and returns the mean number of
    time-steps needed to clean the fraction MIN_COVERAGE of the room.

    The simulation is run with NUM_ROBOTS robots of type ROBOT_TYPE, each with
    speed SPEED, in a room of dimensions WIDTH x HEIGHT.

    num_robots: an int (num_robots > 0)
    speed: a float (speed > 0)
    width: an int (width > 0)
    height: an int (height > 0)
    min_coverage: a float (0 <= min_coverage <= 1.0)
    num_trials: an int (num_trials > 0)
    robot_type: class of robot to be instantiated (e.g. StandardRobot or
                RandomWalkRobot)
    """
    all_steps = []
    for trial in range(num_trials):
        room = RectangularRoom(width, height)
        robots = []
        for robot in range(num_robots):
            robots.append(robot_type(room, speed))
            
        time_step = 1
        while True:
            for robot in robots:
                robot.updatePositionAndClean()
            frac_cleaned = float(room.getNumCleanedTiles())/room.getNumTiles()
            if frac_cleaned >= min_coverage:
                all_steps.append(time_step)
                break
            time_step +=  1
    return float(sum(all_steps))/num_trials

print  runSimulation(1, 1.0, 5, 5, 0.78, 30, StandardRobot)

64.8666666667


In [6]:
"""
VISUALIZING ROBOTS

Note: This part is optional. It is cool and very easy to do, and may also be useful for debugging. Be sure to comment out all 
visualization parts of your code before submitting. We've provided some code to generate animations of your robots as they go 
about cleaning a room. These animations can also help you debug your simulation by helping you to visually determine when things
are going wrong. Here's how to run the visualization:

In your simulation, at the beginning of a trial, insert the following code to start an animation:


anim = ps2_visualize.RobotVisualization(num_robots, width, height)


(Pass in parameters appropriate to the trial, of course.) This will open a new window to display the animation and draw a 
picture of the room. Then, during each time-step, before the robot(s) move, insert the following code to draw a new frame 
of the animation:


anim.update(room, robots)


where room is a RectangularRoom object and robots is a list of Robot objects representing the current state of the room and 
the robots in the room. When the trial is over, call the following method:


anim.done()


The visualization code slows down your simulation so that the animation doesn't zip by too fast (by default, it shows 5 
time-steps every second). Naturally, you will want to avoid running the animation code if you are trying to run many trials 
at once (for example, when you are running the full simulation).

For purposes of debugging your simulation, you can slow down the animation even further. You can do this by changing the call 
to RobotVisualization, as follows:


anim = ps2_visualize.RobotVisualization(num_robots, width, height, delay)


The parameter delay specifies how many seconds the program should pause between frames. The default is 0.2 (that is, 5 frames 
per second). You can increase this value to make the animation slower or decrease it (0.01 is reasonable) to see many robots 
cleaning the room at a faster frame rate.

For problem 5, we will make calls to runSimulation() to get simulation data and plot it. However, you don't want the 
visualization getting in the way. If you choose to do this visualization exercise, before you get started on problem 5 (and 
before you submit your code in submission boxes), make sure to comment the visualization code out of runSimulation().

"""
def runSimulation(num_robots, speed, width, height, min_coverage, num_trials, robot_type, display=False):
    """
    Runs NUM_TRIALS trials of the simulation and returns the mean number of
    time-steps needed to clean the fraction MIN_COVERAGE of the room.

    The simulation is run with NUM_ROBOTS robots of type ROBOT_TYPE, each with
    speed SPEED, in a room of dimensions WIDTH x HEIGHT.

    num_robots: an int (num_robots > 0)
    speed: a float (speed > 0)
    width: an int (width > 0)
    height: an int (height > 0)
    min_coverage: a float (0 <= min_coverage <= 1.0)
    num_trials: an int (num_trials > 0)
    robot_type: class of robot to be instantiated (e.g. StandardRobot or
                RandomWalkRobot)
    """
    all_steps = []
    for trial in range(num_trials):
        if display:
            anim = ps2_visualize.RobotVisualization(num_robots, width, height)
        room = RectangularRoom(width, height)
        robots = []
        for robot in range(num_robots):
            robots.append(robot_type(room, speed))
            
        time_step = 1
        while True:
            if display:
                anim.update(room, robots)
            for robot in robots:
                robot.updatePositionAndClean()
            frac_cleaned = float(room.getNumCleanedTiles())/room.getNumTiles()
            if frac_cleaned >= min_coverage:
                all_steps.append(time_step)
                break
            time_step +=  1
        if display:
            anim.done()
    return float(sum(all_steps))/num_trials

print  runSimulation(5, 1.0, 30, 30, 0.4, 2, StandardRobot)

100.0


In [7]:
"""
Problem 4: RandomWalkRobot Class

iRobot is testing out a new robot design. The proposed new robots differ in that they change direction randomly after every 
time step, rather than just when they run into walls. You have been asked to design a simulation to determine what effect, if 
any, this change has on room cleaning times.

Write a new class RandomWalkRobot that inherits from Robot (like StandardRobot) but implements the new movement strategy. 
RandomWalkRobot should have the same interface as StandardRobot.

Test out your new class. Perform a single trial with the StandardRobot implementation and watch the visualization to make sure 
it is doing the right thing. Once you are satisfied, you can call runSimulation again, passing RandomWalkRobot instead of 
StandardRobot.

Enter your code for classes Robot and RandomWalkRobot below.
"""
class RandomWalkRobot(Robot):
    """
    A RandomWalkRobot is a robot with the "random walk" movement strategy: it
    chooses a new direction at random at the end of each time-step.
    """
    def updatePositionAndClean(self):
        """
        Simulate the passage of a single time-step.

        Move the robot to a new position and mark the tile it is on as having
        been cleaned.
        """
        position = self.getRobotPosition()
        direction = self.getRobotDirection()
        newPosition = position.getNewPosition(direction, self.speed)
        if self.room.isPositionInRoom(newPosition):
            self.setRobotPosition(newPosition)
            self.room.cleanTileAtPosition(newPosition)
        direction = random.randint(0,360)
        self.setRobotDirection(direction)
            
print  runSimulation(1, 1.0, 5, 5, 0.4, 2, RandomWalkRobot)

14.5


In [10]:
"""
Now, you'll use your simulation to answer some questions about the robots' performance.

Problem 5-1A

Examine showPlot1, which takes in the parameters title, x_label, and y_label. Your job is to examine the code and figure out 
what the plot produced by the function tells you. Try calling showPlot1 with appropriate arguments to produce a few plots. 
Then, answer the following 3 questions.

Which of the following would be the best title for the graph?
- 
"""
def showPlot1(title, x_label, y_label):
    """
    What information does the plot produced by this function tell you?
    """
    num_robot_range = range(1, 11)
    times1 = []
    times2 = []
    for num_robots in num_robot_range:
        print "Plotting", num_robots, "robots..."
        times1.append(runSimulation(num_robots, 1.0, 20, 20, 0.8, 20, StandardRobot))
        times2.append(runSimulation(num_robots, 1.0, 20, 20, 0.8, 20, RandomWalkRobot))
    pylab.plot(num_robot_range, times1)
    pylab.plot(num_robot_range, times2)
    pylab.title(title)
    pylab.legend(('StandardRobot', 'RandomWalkRobot'))
    pylab.xlabel(x_label)
    pylab.ylabel(y_label)
    pylab.show()

    
def showPlot2(title, x_label, y_label):
    """
    What information does the plot produced by this function tell you?
    """
    aspect_ratios = []
    times1 = []
    times2 = []
    for width in [10, 20, 25, 50]:
        height = 300/width
        print "Plotting cleaning time for a room of width:", width, "by height:", height
        aspect_ratios.append(float(width) / height)
        times1.append(runSimulation(2, 1.0, width, height, 0.8, 200, StandardRobot))
        times2.append(runSimulation(2, 1.0, width, height, 0.8, 200, RandomWalkRobot))
    pylab.plot(aspect_ratios, times1)
    pylab.plot(aspect_ratios, times2)
    pylab.title(title)
    pylab.legend(('StandardRobot', 'RandomWalkRobot'))
    pylab.xlabel(x_label)
    pylab.ylabel(y_label)
    pylab.show()
    

# === Problem 5
# 1) Write a function call to showPlot1 that generates an appropriately-labeled plot.

title = 'Time It Takes 1 - 10 Robots To Clean 80% Of A Room'
x_label = 'Number of robots'
y_label = 'Time-steps'
showPlot1(title, x_label, y_label)

Plotting 1 robots...
Plotting 2 robots...
Plotting 3 robots...
Plotting 4 robots...
Plotting 5 robots...
Plotting 6 robots...
Plotting 7 robots...
Plotting 8 robots...
Plotting 9 robots...
Plotting 10 robots...


In [11]:
# 2) Write a function call to showPlot2 that generates an appropriately-labeled plot.
title = 'Time It Takes Two Robots To Clean 80% Of Variously Shaped Rooms'
x_label = 'Aspect Ratio'
y_label = 'Time-steps'
showPlot2(title, x_label, y_label)

Plotting cleaning time for a room of width: 10 by height: 30
Plotting cleaning time for a room of width: 20 by height: 15
Plotting cleaning time for a room of width: 25 by height: 12
Plotting cleaning time for a room of width: 50 by height: 6
